## 통계학 실험 5장 과제 

#### 1. 자료를 통한 예제 (ames.csv)

주어진 자료는 Iowa의 도시 Ames의 2006년부터 2010년 사이의 부동산 거래내역 자료이다. 5년 동안 이 지역에서 발생한 총 2930건의 부동산 거래내역이 모두 기록되어 있다. 본 예제에서는 집의 크기를 나타내는 변수인 `Gr.Liv.Area`를 모집단으로 사용하도록 한다.

1-1. 주어진 자료는 전체 부동산에 대한 자료이므로 모집단으로 생각할 수 있다. 거래가 이루어진 전체 부동산의 집의 크기의 평균값 ($\mu$)은 얼마인가? 모분산 ($\sigma^2$)은 얼마인가?

In [28]:
import lzma
import pandas as pd
from IPython.display import Markdown as md

area = pd.read_csv(lzma.open('ames.csv.xz'))['Gr.Liv.Area']

mean = area.mean()
var = area.var()

md(rf'''
- $\mu = {area.mean():.7}$
- $\sigma^2 = {var:.7}$
''')


- $\mu = 1499.69$
- $\sigma^2 = 255539.2$


&nbsp;

1-2. 모집단에서 크기가 60인 랜덤 표본을 선택하자. 모집단 평균에 대한 점추정값은 얼마인가?

In [29]:
sample_size = 60
sample_mean = area.sample(sample_size).mean()

md(rf'''
- $\overline{{X}} = {sample_mean:.7}$
''')


- $\overline{X} = 1570.333$


&nbsp;

1-3. 예제 2에서 선택된 표본을 이용하여 모평균에 대한 95% 신뢰구간을 구해보자. 이 때, 모분산은 예제 1에서 구한 값을 사용하도록 한다. 이 신뢰구간은 모평균을 포함하는가?

In [30]:
from scipy.stats import norm

alpha = (100 - 95)*0.01
z_half_alpha = norm.ppf((1 - alpha)/2 + 0.5)
half_range = z_half_alpha * (var / sample_size) ** 0.5

md(rf'''
- 95% 신뢰구간: $({sample_mean - half_range:.7}, {sample_mean + half_range:.7})$
''')


- 95% 신뢰구간: $(1442.424, 1698.242)$


&nbsp;

1-4. 예제 3과 동일한 과정을 200번 반복하여 서로 다른 신뢰구간 200개를 구해보자. 이 200개의 신뢰구간 중 모평균을 실제로 포함하는 값의 비율은 어떻게 되는지 확인하시오.

In [4]:
total_num = 200
success_num = sum(mean - half_range < area.sample(sample_size).mean() < mean + half_range for _ in range(total_num))

md(rf'''
- 성공 비율: ${success_num / total_num * 100}\%$

신뢰수준과 비슷한 확률로 성공했음을 알 수 있다.
''')


- 성공 비율: $96.5\%$

신뢰수준과 비슷한 확률로 성공했음을 알 수 있다.


&nbsp;

#### 2. 아래 코드를 통해 생성된 함수 `f1`와 `f2`는 $\mu$, $\sigma$, $n$을 차례대로 인수로 받아 $N(\mu$,$\sigma^2)$에서 $n$개의 표본을 랜덤하게 추출하여 각각 순서대로 표본평균, 표본분산을 반환하는 함수이다. 예를 들어 `f1(2,3,4)`는 $N(2,3^2)$에서 4개의 표본을 랜덤하게 추출하여 표본평균을 반환하며, `f2(2,3,4)`는 표본분산을 반환한다.

In [5]:
import numpy as np

def f1(mu, sigma, n):
    sample = norm.rvs(loc=mu, scale=sigma, size=n)
    return np.mean(sample)

def f2(mu, sigma, n):
    sample = norm.rvs(loc=mu, scale=sigma, size=n)
    return (np.std(sample)**2*(n/(n-1))) # '표본분산'이므로 (n/(n-1))을 곱해야 함.

위 함수 f1, f2를 이용하여 다음에 답하시오.

(1) $N(1, 3^2)$에서 10개의 표본을 랜덤하게 추출하여 표본평균을 구하는 시행을 100회 반복할 때, 해당 표본평균들의 표본평균과 표본표준편차를 계산하시오. 

(2) $N(1, 3^2)$에서 1000개의 표본을 랜덤하게 추출하여 표본평균을 구하는 시행을 100회 반복할 때, 해당 표본평균들의 표본평균과 표본표준편차를 계산하시오. (1)과 (2)의 결과를 간략히 비교하시오.

(3) $N(1, 3^2)$에서 10개의 표본을 랜덤하게 추출하여 표본분산을 구하는 시행을 100회 반복할 때, 해당 표본분산들의 표본평균과 표본표준편차를 계산하시오.

(4) $N(1, 3^2)$에서 1000개의 표본을 랜덤하게 추출하여 표본분산을 구하는 시행을 100회 반복할 때, 해당 표본분산들의 표본평균과 표본표준편차를 계산하시오. (3)과 (4)의 결과를 간략히 비교하시오.

In [32]:
def func(func, sample_size: int, description: str) -> str:
    samples = np.array([func(1, 3, sample_size) for _ in range(100)])
    return rf'''- {description} 100개의 표본평균: ${samples.mean():.5}$
- {description} 100개의 표본표준편차: ${(samples.var()*100/99)**0.5:.5}$
'''

md(rf'''
(1)
{func(f1, 10, '크기 10 표본평균')}
(2)
{func(f1, 1000, '크기 1000 표본평균')}

둘의 표본평균은 비슷한 반면, 표본표준편차는 1/10로 줄어들었음을 알 수 있다. 이는 표본평균의 크기가 $n = 100$배로 줄었기때문에 $\frac{{1}}{{\sqrt n}} = \frac{{1}}{{10}}$배로 감소한것이다.

(3)
{func(f2, 10, '크기 10 표본분산')}
(4)
{func(f2, 1000, '크기 1000 표본분산')}

둘의 표본평균은 비슷한 반면, 표본표준편차는 1/10로 줄어들었다.
''')


(1)
- 크기 10 표본평균 100개의 표본평균: $1.0262$
- 크기 10 표본평균 100개의 표본표준편차: $0.97307$

(2)
- 크기 1000 표본평균 100개의 표본평균: $0.99868$
- 크기 1000 표본평균 100개의 표본표준편차: $0.097257$


둘의 표본평균은 비슷한 반면, 표본표준편차는 1/10로 줄어들었음을 알 수 있다. 이는 표본평균의 크기가 $n = 100$배로 줄었기때문에 $\frac{1}{\sqrt n} = \frac{1}{10}$배로 감소한것이다.

(3)
- 크기 10 표본분산 100개의 표본평균: $9.0045$
- 크기 10 표본분산 100개의 표본표준편차: $4.7462$

(4)
- 크기 1000 표본분산 100개의 표본평균: $9.044$
- 크기 1000 표본분산 100개의 표본표준편차: $0.3895$


둘의 표본평균은 비슷한 반면, 표본표준편차는 1/10로 줄어들었다.


&nbsp;

#### 3. 한 공장에서 생성되는 20개 전구의 수명을 조사한 결과가 다음과 같다. 물음에 답하시오

    51, 52, 59, 57, 48, 57, 45, 58, 50, 42, 54, 43, 55, 55, 56, 56, 50, 58, 49, 52

(1) 표본평균과 표본표준편차를 구하시오.

(2) 전구의 수명의 표준편차가 6으로 알려져 있을 때, 공장에서 생성되는 전구의 수명의 평균이 50보다 큰지를 유의수준 5% 이내에서 검정할 때,  
검정통계량의 값과 기각여부를 밝히시오. 

In [40]:
sample = np.array([51, 52, 59, 57, 48, 57, 45, 58, 50, 42, 54, 43, 55, 55, 56, 56, 50, 58, 49, 52])

sample_mean = sample.mean()
sample_std = (sample.var() * 20/19)**0.5

sigma = 6
Z = (sample_mean - 50)/sigma

display(md(rf'''
(1)
- 표본평균: ${sample_mean}$
- 표본표준편차: ${sample_std:.7}$

(2)
- 검정통계량: $Z = {Z:.4}$
- 기각역: $Z \geq {norm.ppf(0.95):.4}$
- $Z = {Z:.4} \ngeq {norm.ppf(0.95):.4}$, 귀무가설이 기각되지 못했음
'''))


(1)
- 표본평균: $52.35$
- 표본표준편차: $5.060424$

(2)
- 검정통계량: $Z = 0.3917$
- 기각역: $Z \geq 1.645$
- $Z = 0.3917 \ngeq 1.645$, 귀무가설이 기각되지 못했음
